In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import sem, ttest_ind
from matplotlib import pyplot as plt
%matplotlib notebook

from re import sub
from decimal import Decimal

In [4]:
company_df = pd.read_csv("company_data.csv")
company_df.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [11]:
#Number of Top 500 S&P Companies in each Industry Sector

company_group = company_df.groupby("Sector").count()
company_group[["Symbol"]]

,Symbol
Sector,
Consumer Discretionary,84
Consumer Staples,34
Energy,32
Financials,68
Health Care,61
Industrials,67
Information Technology,70
Materials,25
Real Estate,33


In [60]:
#how many computer/math employees there are per state

math_emp = "BLS_math_employees.csv"
math_df = pd.read_csv(math_emp, encoding="utf-8")
math_df = math_df.sort_values("Mean_wage_yr", ascending=False)
math_df.head()

,State Name,State,Employ_No,Mean_wage_yr,Median_wage_yr,Employ per 1000 jobs
50,Washington,WA,164830,108660,106350,50.574
8,District of Columbia,DC,42120,108600,107780,59.123
4,California,CA,616270,108190,103310,36.235
49,Virginia,VA,206750,102430,98810,53.940
31,New Jersey,NJ,144240,101390,96480,35.612


In [51]:
fortune_1000 = pd.read_csv("fortune_1000_part.csv")
revenue_int = []
for x in fortune_1000["Revenues ($M)"]:
    revenue_int.append(float(x.replace("$","").replace(",","").replace(" ","")))
fortune_1000["Revenue as Int"] = revenue_int
fortune_1000

,Company,Revenues ($M),Profits ($M),Assets ($M),Employees,Sector,Industry,City,State,Revenue as Int
0,Walmart,"$500,343","$9,862.00","$204,522","2,300,000",Retailing,General Merchandisers,Bentonville,AR,500343.0
1,Exxon Mobil,"$244,363","$19,710.00","$348,691","71,200",Energy,Petroleum Refining,Irving,TX,244363.0
2,Berkshire Hathaway,"$242,137","$44,940.00","$702,095","377,000",Financials,Insurance: Property and Casualty (Stock),Omaha,NE,242137.0
3,Apple,"$229,234","$48,351.00","$375,319","123,000",Technology,"Computers, Office Equipment",Cupertino,CA,229234.0
4,UnitedHealth Group,"$201,159","$10,558.00","$139,058","260,000",Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN,201159.0
5,McKesson,"$198,533","$5,070.00","$60,969","64,500",Wholesalers,Wholesalers: Health Care,SF,CA,198533.0
6,CVS Health,"$184,765","$6,622.00","$95,131","203,000",Health Care,Health Care: Pharmacy and Other Services,Woonsocket,RI,184765.0
7,Amazon.com,"$177,866","$3,033.00","$131,310","566,000",Retailing,Internet Services and Retailing,Seattle,WA,177866.0
8,AT&T,"$160,546","$29,450.00","$444,097","254,000",Telecommunications,Telecommunications,Dallas,TX,160546.0
9,General Motors,"$157,311","($3,864.00)","$212,482","180,000",Motor Vehicles & Parts,Motor Vehicles and Parts,Detroit,MI,157311.0


In [56]:
fortune_group = fortune_1000.groupby("State")
fortune_count = fortune_group["Revenue as Int"].count()
fortune_count

State
AL               3
AR               8
AZ              17
CA             118
CO              19
CT              27
DE               2
FL              35
GA              30
HI               3
IA               6
ID               3
IL              70
IN              17
KS               6
KY               9
LA               3
MA              32
MD              12
ME               1
MI              27
MN              25
MO              23
MS               1
Michigan         1
NC              26
ND               1
NE               8
NH               2
NJ              33
NV               9
NY              93
OH              57
OK              10
OR               5
PA              42
Puerto Rico      2
RI               7
SC               5
TN              23
TX              94
UT               4
VA              38
VT               1
WA              20
WI              22
Name: Revenue as Int, dtype: int64

In [54]:
fortune_mean = fortune_group["Revenue as Int"].mean()
fortune_mean

State
AL              4651.466667
AR             71746.875000
AZ              6149.411765
CA             14922.652542
CO              7440.315789
CT             13847.222222
DE              5681.000000
FL              9397.025714
GA             14390.233333
HI              2433.000000
IA              5963.666667
ID             28144.000000
IL             13901.100000
IN             11762.470588
KS              4072.666667
KY              9284.666667
LA             10506.333333
MA             12412.531250
MD              9778.666667
ME              1969.000000
MI             19178.703704
MN             22332.920000
MO             11784.434783
MS              3342.000000
Michigan       62683.000000
NC             12517.923077
ND              4444.000000
NE             36616.375000
NH              2883.500000
NJ             13188.909091
NV              5423.888889
NY             16296.639785
OH             13076.494737
OK              6944.700000
OR             10216.200000
PA            